# Assignment 3 - CS3920

In [243]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, RobustScaler
from sklearn.base import BaseEstimator

import numpy as np

In [244]:
CV: int = 3
RANDOM_STATE: int = 3103

# 1 - Load Datasets

## Loading Wine Dataset

In [245]:
wine: np.ndarray = load_wine()
X_wine: np.ndarray = wine.data
y_wine: np.ndarray = wine.target

## Loading USPS Dataset

In [246]:
DATA_SPLIT: int = 10 # percentage of data to be used 

In [247]:
zip_train: np.ndarray = np.genfromtxt('zip.train', delimiter=' ')
zip_train = zip_train[::DATA_SPLIT]

zip_test: np.ndarray = np.genfromtxt('zip.test', delimiter=' ')
zip_test = zip_test[::DATA_SPLIT]

zip_data: np.ndarray = np.concatenate((zip_train, zip_test), axis=0)
zip_data = zip_data[::DATA_SPLIT]

X_zip: np.ndarray = zip_data[:, 1:]
y_zip: np.ndarray = zip_data[:, 0]

In [248]:
del zip_train, zip_test

# 2 - Split Datasets 

In [249]:
def split(X: np.ndarray, y: np.ndarray) -> tuple[np.ndarray[float]]:
	return train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

## Splitting Wine Dataset

In [250]:
# X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(X_wine, y_wine, test_size=0.3, random_state=RANDOM_STATE)
X_wine_train, X_wine_test, y_wine_train, y_wine_test = split(X_wine, y_wine)

## Splitting Zip Dataset

In [251]:
# X_zip_train, X_zip_test, y_zip_train, y_zip_test = train_test_split(X_zip, y_zip, test_size=0.3, random_state=RANDOM_STATE)
X_zip_train, X_zip_test, y_zip_train, y_zip_test = split(X_zip, y_zip)

# 3 - Cross Validation

## Cross Validate Wine

In [252]:
svm: SVC = SVC()
scores_wine: np.ndarray[float] = cross_val_score(svm, X_wine_train, y_wine_train, cv=CV)
print("Cross-validation scores: ", scores_wine)
print("Average cross-validation score: ", scores_wine.mean())

Cross-validation scores:  [0.64285714 0.63414634 0.68292683]
Average cross-validation score:  0.6533101045296168


## Cross Validate Zip

In [253]:
score_zip: np.ndarray[float] = cross_val_score(svm, X_zip_train, y_zip_train, cv=CV)
print("Cross-validation scores: ", score_zip)
print("Average cross-validation score: ", score_zip.mean())

Cross-validation scores:  [0.63636364 0.72727273 0.61904762]
Average cross-validation score:  0.660894660894661


# 4 - Test Error Rate of `SVM`

## Test Wine

In [254]:
svm.fit(X_wine_train, y_wine_train)
print("Test error rate: ", 1 - svm.score(X_wine_test, y_wine_test))

Test error rate:  0.2777777777777778


## Test Zip

In [255]:
svm.fit(X_zip_train, y_zip_train)
print("Test error rate: ", 1 - svm.score(X_zip_test, y_zip_test))

Test error rate:  0.4482758620689655


# 5 - Pipeline

In [256]:
param_grid: dict[str, list[float]] = {
	'svc__C': [0.001, 0.01, 0.1, 1, 10, 100], 
	'svc__gamma': [0.001, 0.01, 0.1, 1, 10, 100]
}

## Wine Dataset

In [257]:
class Super_Pipeline:
	def __init__ (self, param_grid: dict[str, list[float]], scaler: BaseEstimator, X_train: np.ndarray[float], y_train: np.ndarray[float], X_test: np.ndarray[float], y_test: np.ndarray[float]):
		self.pipeline: Pipeline = make_pipeline(scaler(), SVC())
		self.grid: GridSearchCV = GridSearchCV(self.pipeline, param_grid, cv=CV)
		self.grid.fit(X_train, y_train)
		self.X_train: np.ndarray[float] = X_train
		self.y_train: np.ndarray[float] = y_train
		self.X_test: np.ndarray[float] = X_test
		self.y_test: np.ndarray[float] = y_test
		
	@property
	def best_score(self) -> float:
		return self.grid.best_score_

	@property
	def best_params(self) -> dict[str, float]:
		return self.grid.best_params_

	@property
	def test_score(self) -> float:
		return self.grid.score(self.X_test, self.y_test)

	@property
	def error_rate(self) -> float:
		return 1 - self.best_score

	def __str__(self) -> str:
		best_score: str = f"Best cross-validation score: {self.best_score}"
		best_params: str = f"Best parameters: {self.best_params}"
		test_score: str = f"Test set score: {self.test_score}"
		error_rate: str = f"Test set error rate: {self.error_rate}"
		return f"{best_score}\n{best_params}\n{test_score}\n{error_rate}"
		# return f"{best_score}\n{best_params}\n{error_rate}"
		
		# return (f"""
		# Best cross-validation score: {self.best_score} 
		# Best parameters: {self.best_params}
		# Test set score: {self.test_score}
		# Test set error rate: {self.error_rate}
		# """)
		

### MinMax Scaler

In [258]:
wine_super_pipe_minmax: Super_Pipeline = Super_Pipeline(param_grid, MinMaxScaler, X_wine_train, y_wine_train, X_wine_test, y_wine_test)
print(wine_super_pipe_minmax)

Best cross-validation score: 0.991869918699187
Best parameters: {'svc__C': 10, 'svc__gamma': 0.1}
Test set score: 0.9814814814814815
Test set error rate: 0.008130081300813052


### Standard Scaler

In [259]:
wine_super_pipe_standard: Super_Pipeline = Super_Pipeline(param_grid, StandardScaler, X_wine_train, y_wine_train, X_wine_test, y_wine_test)
print(wine_super_pipe_standard)

Best cross-validation score: 0.983933410762679
Best parameters: {'svc__C': 100, 'svc__gamma': 0.001}
Test set score: 0.9814814814814815
Test set error rate: 0.01606658923732096


### Normalizer

In [260]:
wine_super_pipe_normalizer: Super_Pipeline = Super_Pipeline(param_grid, Normalizer, X_wine_train, y_wine_train, X_wine_test, y_wine_test)
print(wine_super_pipe_normalizer)

Best cross-validation score: 0.927022841656988
Best parameters: {'svc__C': 100, 'svc__gamma': 100}
Test set score: 0.9629629629629629
Test set error rate: 0.072977158343012


### Robust Scaler

In [261]:
wine_super_pipe_robust: Super_Pipeline = Super_Pipeline(param_grid, RobustScaler, X_wine_train, y_wine_train, X_wine_test, y_wine_test)
print(wine_super_pipe_robust)

Best cross-validation score: 0.983739837398374
Best parameters: {'svc__C': 10, 'svc__gamma': 0.01}
Test set score: 0.9814814814814815
Test set error rate: 0.016260162601625994


## Zip Dataset

### MinMax Scaler

In [263]:
zip_super_pipe_minmax: Super_Pipeline = Super_Pipeline(param_grid, MinMaxScaler, X_zip_train, y_zip_train, X_zip_test, y_zip_test)
print(zip_super_pipe_minmax)

Best cross-validation score: 0.7373737373737373
Best parameters: {'svc__C': 100, 'svc__gamma': 0.001}
Test set score: 0.5862068965517241
Test set error rate: 0.26262626262626265


### Standard Scaler

In [265]:
zip_super_pipe_standard: Super_Pipeline = Super_Pipeline(param_grid, StandardScaler, X_zip_train, y_zip_train, X_zip_test, y_zip_test)
print(zip_super_pipe_standard)

Best cross-validation score: 0.7070707070707071
Best parameters: {'svc__C': 10, 'svc__gamma': 0.001}
Test set score: 0.5517241379310345
Test set error rate: 0.29292929292929293


### Normalizer

In [278]:
zip_super_pipe_robust: Super_Pipeline = Super_Pipeline(param_grid, RobustScaler, X_zip_train, y_zip_train, X_zip_test, y_zip_test)
print(zip_super_pipe_robust)

Best cross-validation score: 0.4437229437229437
Best parameters: {'svc__C': 100, 'svc__gamma': 0.001}
Test set score: 0.5862068965517241
Test set error rate: 0.5562770562770563


### Robust Scaler

In [279]:
zip_super_pipe_normalizer: Super_Pipeline = Super_Pipeline(param_grid, Normalizer, X_zip_train, y_zip_train, X_zip_test, y_zip_test)
print(zip_super_pipe_normalizer)

Best cross-validation score: 0.7532467532467533
Best parameters: {'svc__C': 100, 'svc__gamma': 0.1}
Test set score: 0.5862068965517241
Test set error rate: 0.24675324675324672


# 6 - Predict Test Labels

## Wine Dataset

### MinMax Scaler

In [268]:
grid_wine_minmax.fit(X_wine_train, y_wine_train)
prediction_wine_minmax: np.ndarray = grid_wine_minmax.predict(X_wine_test)
print("Test Set Score: ", grid_wine_minmax.score(X_wine_test, y_wine_test))
# print("Test Set Score: ", np.mean(prediction_wine_minmax == y_wine_test))

Test Set Score:  0.9814814814814815


### Standard Scaler

In [269]:
grid_wine_standard.fit(X_wine_train, y_wine_train)
prediction_wine_standard: np.ndarray = grid_wine_standard.predict(X_wine_test)
print("Test Set Score: ", grid_wine_standard.score(X_wine_test, y_wine_test))

Test Set Score:  0.9814814814814815


### Normalizer

In [270]:
grid_wine_normalizer.fit(X_wine_train, y_wine_train)
prediction_wine_normalizer: np.ndarray = grid_wine_normalizer.predict(X_wine_test)
print("Test Set Score: ", grid_wine_normalizer.score(X_wine_test, y_wine_test))

Test Set Score:  0.9629629629629629


### Robust Scalar

In [271]:
grid_wine_robust.fit(X_wine_train, y_wine_train)
prediction_wine_robust: np.ndarray = grid_wine_robust.predict(X_wine_test)
print("Test Set Score: ", grid_wine_robust.score(X_wine_test, y_wine_test))

Test Set Score:  0.9814814814814815


## Zip Dataset

### MinMax Scaler

In [272]:
grid_zip_minmax.fit(X_zip_train, y_zip_train)
prediction_zip_minmax: np.ndarray = grid_zip_minmax.predict(X_zip_test)
print("Test Set Score: ", grid_zip_minmax.score(X_zip_test, y_zip_test))

Test Set Score:  0.5862068965517241


### Standard Scaler

In [273]:
grid_zip_standard.fit(X_zip_train, y_zip_train)
prediction_zip_standard: np.ndarray = grid_zip_standard.predict(X_zip_test)
print("Test Set Score: ", grid_zip_standard.score(X_zip_test, y_zip_test))

Test Set Score:  0.5517241379310345


### Normalizer

In [274]:
grid_zip_normalizer.fit(X_zip_train, y_zip_train)
prediction_zip_normalizer: np.ndarray = grid_zip_normalizer.predict(X_zip_test)
print("Test Set Score: ", grid_zip_normalizer.score(X_zip_test, y_zip_test))

Test Set Score:  0.5862068965517241


### Robust Scaler

In [275]:
grid_zip_robust.fit(X_zip_train, y_zip_train)
prediction_zip_robust: np.ndarray = grid_zip_robust.predict(X_zip_test)
print("Test Set Score: ", grid_zip_robust.score(X_zip_test, y_zip_test))

Test Set Score:  0.5862068965517241
